############# SHOCK ADAPTATION FOOD SUPPLY MODEL #############


 SOPHIA BAUM - 2024

In [1]:
tau = 10                            # number of iterations
compensation=True                 # turn adaptation on
limit_abs_sim=1000                  # event limits
limit_rel_sim=0.26
limit_dev_sim=0.32

In [2]:
### IMPORT ###
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from pandas import IndexSlice as idx

In [4]:
import scipy.io as io
import scipy.sparse as sprs

In [5]:
import numpy as np

In [6]:
import os
print(os.getcwd())

/Users/valentinbast/uni/compelx_sys_modelling/project/adaptive_food_supply_network-main/csm_project_


## PARAMETERS ###

In [7]:
input_folder = 'input/'                 # folder with parameters
output_folder =  'results/'             # folder to write results to
losses = 'evaluation/'                  # folder to store the refined results to

In [8]:
a_shock = 'IND'

In [9]:
a_frame = pd.read_csv(input_folder+'a_frame.csv')
area_value = a_frame.loc[a_frame['code'] == a_shock, 'code'].values[0]

## LOADING DATA ###

In [10]:
# Load information
io_codes = pd.read_csv(input_folder+'io_codes_alph.csv').drop('Unnamed: 0', axis = 1)
su_codes = pd.read_csv(input_folder+'su_codes_alph.csv').drop('Unnamed: 0', axis = 1)

In [11]:
# Create single indexes
areas = np.array(sorted(set(io_codes['area'])))
items = np.array(sorted(set(io_codes['item'])))
processes = np.array(sorted(set(su_codes['proc'])))

In [12]:
# Create multi indexes
ai_index = pd.MultiIndex.from_product([areas, items])
ap_index = pd.MultiIndex.from_product([areas, items])

In [13]:
# Load  further information on countries
a_frame = pd.read_csv(input_folder+'a_frame.csv')
# Set 'code' as the index
a_frame.set_index('code', inplace=True)

# Check if a_shock is in the index
print(f"a_shock: {a_shock}")
print(f"Is a_shock in a_frame index? {a_shock in a_frame.index}")
print(a_frame.index)
print(f"a_shock: {a_shock}, type: {type(a_shock)}")

a_shock: IND
Is a_shock in a_frame index? True
Index(['MAR', 'DZA', 'LBY', 'TUN', 'SDN', 'EGY', 'SEN', 'ETH', 'SWZ', 'SLE',
       ...
       'KIR', 'PYF', 'WSM', 'BLX', 'CSK', 'ANT', 'ROW', 'SCG', 'SUN', 'YUG'],
      dtype='object', name='code', length=192)
a_shock: IND, type: <class 'str'>


In [14]:
# Load the result of the shocked simulation
X = pd.read_csv(output_folder+'base.csv', index_col=[0,1], header=[0])
XS_comp = pd.read_csv(output_folder + area_value + '_comp.csv', index_col=[0, 1], header=[0, 1])
XS_no_comp = pd.read_csv(output_folder + a_shock + '_no_comp.csv', index_col=[0, 1], header=[0, 1])

## COMPUTATIONS ###

In [15]:
# Compute relative loss
RL_no_comp = XS_no_comp.copy()
RL_comp = XS_comp.copy()
for col in XS_no_comp.columns:
    RL_no_comp[col] = ((X['base'] - RL_no_comp[col])/X['base']).fillna(0)
    RL_comp[col] = ((X['base'] - RL_comp[col])/X['base']).fillna(0)  
RL_no_comp[RL_no_comp < -1] = -1
RL_comp[RL_comp < -1] = -1

In [16]:
# Setup a dataframe for the relative loss
RL_no_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
RL_no_comp.columns.names = ['a_shock','i_shock']
RL_no_comp.index.names = ['a_receive','i_receive'] 
RL_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
RL_comp.columns.names = ['a_shock','i_shock']
RL_comp.index.names = ['a_receive','i_receive'] 

In [17]:
# Save
RL_no_comp.to_csv(losses + 'RL-' + a_shock + '_no_comp.csv')
RL_comp.to_csv(losses + 'RL-' + a_shock + '_comp.csv')

In [18]:
# Compute absolute loss
AL_no_comp = XS_no_comp.copy()
AL_comp = XS_comp.copy()
for col in XS_no_comp.columns:
    AL_no_comp[col] = X['base'] - XS_no_comp[col]
    AL_comp[col] = X['base'] - XS_comp[col]

In [19]:
# Setup a dataframe for the absolute loss
AL_no_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
AL_no_comp.columns.names = ['a_shock','i_shock']
AL_no_comp.index.names = ['a_receive','i_receive'] 
AL_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
AL_comp.columns.names = ['a_shock','i_shock']
AL_comp.index.names = ['a_receive','i_receive']  

In [20]:
# Save
AL_no_comp.to_csv(losses + 'AL-' + a_shock + '_no_comp.csv')
AL_comp.to_csv(losses + 'AL-' + a_shock + '_comp.csv')

In [21]:
# Step 1: Skip resetting the index since 'a_receive' is already a column
AL_no_comp_flat = AL_no_comp.copy()
AL_comp_flat = AL_comp.copy()

# Step 2: Flatten column MultiIndex only if necessary
if isinstance(AL_no_comp_flat.columns, pd.MultiIndex):
    AL_no_comp_flat.columns = ['_'.join(filter(None, col)) for col in AL_no_comp_flat.columns]
if isinstance(AL_comp_flat.columns, pd.MultiIndex):
    AL_comp_flat.columns = ['_'.join(filter(None, col)) for col in AL_comp_flat.columns]

In [24]:
print("AL_no_comp_flat columns:", AL_no_comp_flat.columns)
print("a_frame columns:", a_frame.columns)

AL_no_comp_flat columns: Index(['IND_Abaca', 'IND_Alcohol, Non-Food', 'IND_Apples and products',
       'IND_Asses', 'IND_Bananas', 'IND_Barley and products', 'IND_Beans',
       'IND_Beer', 'IND_Beverages, Alcoholic', 'IND_Beverages, Fermented',
       ...
       'IND_Sweet potatoes', 'IND_Sweeteners, Other',
       'IND_Tea (including mate)', 'IND_Tobacco', 'IND_Tomatoes and products',
       'IND_Vegetables, Other', 'IND_Wheat and products', 'IND_Wine',
       'IND_Wool (Clean Eq.)', 'IND_Yams'],
      dtype='object', length=123)
a_frame columns: Index(['area', 'index', 'region', 'color', 'lat', 'lng', 'population'], dtype='object')


In [25]:
# Add area as a column if it’s currently the index
if AL_no_comp.index.names[0] == 'a_receive':
    AL_no_comp = AL_no_comp.reset_index()

# Flatten and prep
AL_no_comp_flat = AL_no_comp.copy()
if isinstance(AL_no_comp_flat.columns, pd.MultiIndex):
    AL_no_comp_flat.columns = ['_'.join(filter(None, col)) for col in AL_no_comp_flat.columns]

# Merge on 'a_receive' (formerly index) and 'area'
AL_no_comp_pc = AL_no_comp_flat.merge(
    a_frame[['area', 'population']],
    left_on='a_receive',
    right_on='area'
)

In [28]:
# --- STEP 1: Ensure 'a_receive' is a column (reset index if needed)
if AL_no_comp.index.names[0] == 'a_receive':
    AL_no_comp = AL_no_comp.reset_index()
if AL_comp.index.names[0] == 'a_receive':
    AL_comp = AL_comp.reset_index()

# --- STEP 2: Flatten column MultiIndex if needed
AL_no_comp_flat = AL_no_comp.copy()
AL_comp_flat = AL_comp.copy()

if isinstance(AL_no_comp_flat.columns, pd.MultiIndex):
    AL_no_comp_flat.columns = ['_'.join(filter(None, col)) for col in AL_no_comp_flat.columns]
if isinstance(AL_comp_flat.columns, pd.MultiIndex):
    AL_comp_flat.columns = ['_'.join(filter(None, col)) for col in AL_comp_flat.columns]

# --- STEP 3: Merge population info from a_frame
AL_no_comp_pc = AL_no_comp_flat.merge(
    a_frame[['area', 'population']],
    left_on='a_receive',
    right_on='area',
    how='left'  # or 'inner' if you're sure all areas match
)

AL_comp_pc = AL_comp_flat.merge(
    a_frame[['area', 'population']],
    left_on='a_receive',
    right_on='area',
    how='left'
)

# --- STEP 4: Normalize per capita for all 'IND_' columns
for col in AL_no_comp_pc.columns:
    if col.startswith('IND_'):
        AL_no_comp_pc[col] = AL_no_comp_pc[col] / AL_no_comp_pc['population']

for col in AL_comp_pc.columns:
    if col.startswith('IND_'):
        AL_comp_pc[col] = AL_comp_pc[col] / AL_comp_pc['population']

In [29]:
# Normalize per capita only for food item columns
for col in AL_no_comp_pc.columns:
    if col.startswith('IND_'):
        AL_no_comp_pc[col] = AL_no_comp_pc[col] / AL_no_comp_pc['population']

for col in AL_comp_pc.columns:
    if col.startswith('IND_'):
        AL_comp_pc[col] = AL_comp_pc[col] / AL_comp_pc['population']

In [30]:
AL_no_comp_pc.loc[:, AL_no_comp_pc.columns != 'population'] *= 1000
AL_comp_pc.loc[:, AL_comp_pc.columns != 'population'] *= 1000

In [31]:
AL_no_comp_pc.to_csv(losses + 'AL-' + str(a_frame.loc[a_shock, 'index']) + '_no_comp.csv')
AL_comp_pc.to_csv(losses + 'AL-' + str(a_frame.loc[a_shock, 'index']) + '_comp.csv')